In [1]:
import requests

keywords = []
outputs = [] # in title, url, body, len
countries = []
sent = []
time = []
def get_wikipedia_link(location_name):
    base_url = "http://en.wikipedia.org/wiki/"
    formatted_location = location_name.replace(" ", "_")  # Remplacer les espaces par des underscores
    return f"{base_url}{formatted_location}"


while True: 
  keyword = input("Entrez des keywords")
  if keyword != "fin":
    keywords.append(keyword)
  else:
    break

while True : 
    output = input ("Entrer des formats de sortie parmi title, url, body, len")
    if output != "fin":
        outputs.append(output)
    else:
        break

time_input = input("Entrez l'intervalle de temps (format: YYYY-MM-DD YYYY-MM-DD), appuyez sur Entrée si non spécifié: ")
if time_input.strip():  # Check if the input is not empty
    start_date, end_date = map(str.strip, time_input.split())
    time = [start_date, end_date]

sent_input = input("Entrez l'intervalle de sentiments compris dans -1,1 (format : x y avec y>x) , appuyez sur Entrée si non spécifié: ")
if sent_input.strip():  # Check if the input is not empty
    min_sent, max_sent = map(str.strip, sent_input.split())
    sent = [float(min_sent), float(max_sent)]

while True:
    country = input("Entrez les pays d'origines (en Anglais)")
    if country != "fin":
        countries.append(get_wikipedia_link(country))
    else:
        break
        
keywordLoc = input ("Où localiser les keywords ? Taper title, body ou body,title")

r = requests.get("http://eventregistry.org/api/v1/article/getArticles", {
    "action": "getArticles",
    "keyword": keywords,
    "articlesPage": 1,
    "articlesCount": 100,
    "articlesSortBy": "date",
    "articlesSortByAsc": False,
    "articlesArticleBodyLen": -1,
    "resultType": "articles",
    "dataType": [
        "news",
        "pr"
    ],
    "apiKey": "562eab60-8961-4906-82d9-6a5cbd74589c",
    "forceMaxDataTimeWindow": 31,
    "dateStart": time[0] if len(time)>0 else None,
    "dateEnd": time[1] if len(time)>0 else None,
    "sourceLocationUri": countries if len(countries)>0 else None,
    "minSentiment" : min_sent if len(sent)>0 else None,
    "maxSentiment" : max_sent if len(sent)>0 else None,
    "keywordLoc" : keywordLoc
}, headers={"content-type": "application/json"})

articles = r.json()["articles"]["results"]
if "len" in outputs : 
    print("nombre d'articles" , len(articles))
    outputs.remove("len")

with open("resultat.txt", "w") as f:
    for article in articles:
        for x in outputs : 
            f.write(article[x] + "\n")
        f.write("\n")

Entrez des keywords gaz
Entrez des keywords Russie
Entrez des keywords Ukraine
Entrez des keywords fin
Entrer des formats de sortie parmi title, url, body, len title
Entrer des formats de sortie parmi title, url, body, len url
Entrer des formats de sortie parmi title, url, body, len len
Entrer des formats de sortie parmi title, url, body, len fin
Entrez l'intervalle de temps (format: YYYY-MM-DD YYYY-MM-DD), appuyez sur Entrée si non spécifié:  2024-01-01 2024-03-01
Entrez l'intervalle de sentiments compris dans -1,1 (format : x y avec y>x) , appuyez sur Entrée si non spécifié:  
Entrez les pays d'origines (en Anglais) France
Entrez les pays d'origines (en Anglais) fin
Où localiser les keywords ? Taper title, body ou body,title title


nombre d'articles 1


In [ ]:
import tkinter as tk
import requests

def get_wikipedia_link(location_name):
    base_url = "http://en.wikipedia.org/wiki/"
    formatted_location = location_name.replace(" ", "_")
    return f"{base_url}{formatted_location}"

def generate_results(keywords, outputs = ['title','url','body'], time = [], sent = [], countries = [], keyword_loc = 'body'):
    global articles
    r = requests.get("http://eventregistry.org/api/v1/article/getArticles", {
        "action": "getArticles",
        "keyword": keywords,
        "articlesPage": 1,
        "articlesCount": 100,
        "articlesSortBy": "date",
        "articlesSortByAsc": False,
        "articlesArticleBodyLen": -1,
        "resultType": "articles",
        "dataType": [
            "news",
            "pr"
        ],
        "apiKey": "562eab60-8961-4906-82d9-6a5cbd74589c",
        "forceMaxDataTimeWindow": 31,
        "dateStart": time[0] if len(time)>0 else None,
        "dateEnd": time[1] if len(time)>0 else None,
        "sourceLocationUri": [get_wikipedia_link(country) for country in countries] if len(countries)>0 else None,
        "minSentiment" : sent[0] if len(sent)>0 else None,
        "maxSentiment" : sent[1] if len(sent)>0 else None,
        "keywordLoc" : keyword_loc
    }, headers={"content-type": "application/json"})
    
    articles = r.json()["articles"]["results"]
    output_file_path = "resultats.txt"  
    
    with open(output_file_path, "w") as f:
        for article in articles:
            for x in outputs : 
                f.write(article[x] + "\n")
            f.write("\n")

    return output_file_path

def display_results(file_path):
    with open(file_path, "r") as f:
        content = f.read()
    result_window = tk.Toplevel(app)
    result_window.title("Résultats")
    result_text = tk.Text(result_window)
    result_text.insert(tk.END, content)
    result_text.pack()

def on_results_button_click(keywords_entry, outputs_entry, time_entry, sent_entry, countries_entry, keyword_loc_entry):
    parameters = []
    values = []
    if keywords_entry.get() : 
        parameters.append('keywords')
        values.append(keywords_entry.get().split(','))
    if outputs_entry.get() : 
        parameters.append('outputs')
        values.append(outputs_entry.get().split(','))
    if time_entry.get() : 
        parameters.append('time')
        values.append(time_entry.get().split())
    if sent_entry.get() : 
        parameters.append('sent')
        values.append([float(min_sent_entry.get()), float(max_sent_entry.get())])
    if countries_entry.get() : 
        parameters.append('countries')
        values.append(countries_entry.get().split(','))
    if keyword_loc_entry.get() : 
        parameters.append ('keyword_loc')
        values.append(keyword_loc_entry.get())
    arguments = dict(zip(parameters, values))
    file_path = generate_results(**arguments)
    display_results(file_path)  # Display the content in a new window

# Interface utilisateur
app = tk.Tk()
app.title("Application de Collecte d'Informations")

def interface_utilisateur() :
    global keywords_entry, outputs_entry, time_entry, sent_entry, countries_entry, keyword_loc_entry
    # Widgets pour les champs de saisie
    tk.Label(app, text="Keywords:").pack()
    keywords_entry = tk.Entry(app)
    keywords_entry.pack()
    
    tk.Label(app, text="Outputs:").pack()
    outputs_entry = tk.Entry(app)
    outputs_entry.pack()
    
    tk.Label(app, text="Time (YYYY-MM-DD YYYY-MM-DD):").pack()
    time_entry = tk.Entry(app)
    time_entry.pack()
    
    tk.Label(app, text="Sentiment (x y):").pack()
    sent_entry = tk.Entry(app)
    sent_entry.pack()
    
    tk.Label(app, text="Countries:").pack()
    countries_entry = tk.Entry(app)
    countries_entry.pack()
    
    tk.Label(app, text="Keyword Location:").pack()
    keyword_loc_entry = tk.Entry(app)
    keyword_loc_entry.pack()

    return keywords_entry, outputs_entry, time_entry, sent_entry, countries_entry, keyword_loc_entry

def affichage_text() : 
    # Bouton pour générer les résultats
    results_button = tk.Button(app, text="Résultats", command=lambda: on_results_button_click(*entries))
    results_button.pack()

entries = interface_utilisateur()
affichage_text()
app.mainloop()